In [23]:
import pandas as pd

# Define the file path
file_path = '/Users/ivanquackenbush/Documents/Blast/Steven_Schneider.csv'

# Load the CSV data into a DataFrame, skipping the first 8 lines of metadata
df = pd.read_csv(file_path, skiprows=8)

# Display the DataFrame
df.head()


,Date,Equipment,Handedness,Swing Details,Plane Score,Connection Score,Rotation Score,Bat Speed (mph),Rotational Acceleration (g),On Plane Efficiency (%),Attack Angle (deg),Early Connection (deg),Connection at Impact (deg),Vertical Bat Angle (deg),Power (kW),Time to Contact (sec),Peak Hand Speed (mph),Exit Velocity (mph),Launch Angle (deg),Estimated Distance (feet)
0,"Sep 16, 2024 11:10:10 am",mach,Left,Pitching Machine,56,40,36,66.9,7.2,73,11,120,98,-35,3.32,0.17,19.1,NaN,NaN,NaN
1,"Sep 16, 2024 11:10:04 am",mach,Left,Pitching Machine,60,54,43,68.5,10.0,77,10,106,85,-35,3.74,0.16,19.7,NaN,NaN,NaN
2,"Sep 16, 2024 11:08:45 am",mach,Right,Pitching Machine,54,61,39,71.7,8.4,71,14,94,80,-33,4.05,0.16,19.3,NaN,NaN,NaN
3,"Sep 16, 2024 11:08:39 am",mach,Right,Pitching Machine,39,55,44,74.6,10.6,56,12,100,81,-33,4.67,0.15,18.6,NaN,NaN,NaN
4,"Sep 16, 2024 11:08:33 am",mach,Right,Pitching Machine,40,50,44,74.8,10.3,56,16,102,78,-29,4.45,0.16,18.7,NaN,NaN,NaN


## Weekley Progress

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.dates import DateFormatter
import os

# Load the CSV data into a DataFrame, skipping the first 8 lines of metadata
file_path = '/Users/ivanquackenbush/Documents/Blast/Steven_Schneider.csv'
df = pd.read_csv(file_path, skiprows=8)

# Extract the base name of the CSV file to use in the PDF title
csv_name = os.path.basename(file_path).replace('.csv', '_updates')

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Filter data from September 2nd onwards
df = df[df['Date'] >= '2024-09-02']

# Group by week (Monday to Sunday) and calculate necessary averages
weekly_data = df.resample('W-Mon', on='Date').agg({
    'Bat Speed (mph)': ['max', 'mean'],
    'Rotational Acceleration (g)': ['max', 'mean'],
    'On Plane Efficiency (%)': ['mean'],
    'Vertical Bat Angle (deg)': ['mean'],
    'Attack Angle (deg)': ['mean'],
    'Early Connection (deg)': ['mean'],
    'Connection at Impact (deg)': ['mean']
}).reset_index()

# Flatten column names
weekly_data.columns = ['Date', 'Max Bat Speed', 'Avg Bat Speed', 'Max Rotational Acceleration', 'Avg Rotational Acceleration', 'Avg On Plane Efficiency', 'Avg Vertical Bat Angle', 'Avg Attack Angle', 'Avg Early Connection', 'Avg Connection at Impact']

# Calculate the percentage change for bat speed, rotational acceleration, and on plane efficiency
weekly_data['Max Bat Speed Change'] = weekly_data['Max Bat Speed'].pct_change() * 100
weekly_data['Avg Bat Speed Change'] = weekly_data['Avg Bat Speed'].pct_change() * 100
weekly_data['Max Rot Acc Change'] = weekly_data['Max Rotational Acceleration'].pct_change() * 100
weekly_data['Avg Rot Acc Change'] = weekly_data['Avg Rotational Acceleration'].pct_change() * 100
weekly_data['Avg On Plane Eff Change'] = weekly_data['Avg On Plane Efficiency'].pct_change() * 100

# Create the PDF file
pdf_path = f"/Users/ivanquackenbush/Documents/Blast/{csv_name}.pdf"
with PdfPages(pdf_path) as pdf:

    # Create a new figure for the title page
    fig_title, ax_title = plt.subplots(figsize=(10, 1))

    # Set the title as the CSV file name (or any custom title)
    ax_title.text(0.5, 0.5, f'Report for {os.path.basename(file_path).replace(".csv", "")}', fontsize=18, ha='center', va='center')

    # Remove the axes
    ax_title.axis('off')

    # Save the title page as the first page in the PDF
    pdf.savefig(fig_title)
    plt.close(fig_title)

    # Create the figure and axes for six plots
    fig, ax = plt.subplots(6, 1, figsize=(10, 30))

    # Plot for Bat Speed
    ax[0].plot(weekly_data['Date'], weekly_data['Max Bat Speed'], label='Max Bat Speed', color='blue', marker='o')
    ax[0].plot(weekly_data['Date'], weekly_data['Avg Bat Speed'], label='Avg Bat Speed', color='orange', marker='s')

    for i, row in weekly_data.iterrows():
        color = 'green' if row['Max Bat Speed Change'] >= 2 else ('red' if row['Max Bat Speed Change'] <= -2 else 'blue')
        ax[0].plot(row['Date'], row['Max Bat Speed'], 'o', color=color)
        ax[0].text(row['Date'], row['Max Bat Speed'], f"{row['Max Bat Speed']:.1f}", fontsize=9, ha='center', va='bottom')

        color = 'green' if row['Avg Bat Speed Change'] >= 2 else ('red' if row['Avg Bat Speed Change'] <= -2 else 'orange')
        ax[0].plot(row['Date'], row['Avg Bat Speed'], 's', color=color)
        ax[0].text(row['Date'], row['Avg Bat Speed'], f"{row['Avg Bat Speed']:.1f}", fontsize=9, ha='center', va='bottom')

    ax[0].set_title('Weekly Bat Speed Analysis (Max and Average)')
    ax[0].set_xlabel('Week')
    ax[0].set_ylabel('Bat Speed (mph)')
    ax[0].legend()

    # Plot for Rotational Acceleration
    ax[1].plot(weekly_data['Date'], weekly_data['Max Rotational Acceleration'], label='Max Rotational Acceleration', color='blue', marker='o')
    ax[1].plot(weekly_data['Date'], weekly_data['Avg Rotational Acceleration'], label='Avg Rotational Acceleration', color='orange', marker='s')

    for i, row in weekly_data.iterrows():
        color = 'green' if row['Max Rot Acc Change'] >= 2 else ('red' if row['Max Rot Acc Change'] <= -2 else 'blue')
        ax[1].plot(row['Date'], row['Max Rotational Acceleration'], 'o', color=color)
        ax[1].text(row['Date'], row['Max Rotational Acceleration'], f"{row['Max Rotational Acceleration']:.1f}", fontsize=9, ha='center', va='bottom')

        color = 'green' if row['Avg Rot Acc Change'] >= 2 else ('red' if row['Avg Rot Acc Change'] <= -2 else 'orange')
        ax[1].plot(row['Date'], row['Avg Rotational Acceleration'], 's', color=color)
        ax[1].text(row['Date'], row['Avg Rotational Acceleration'], f"{row['Avg Rotational Acceleration']:.1f}", fontsize=9, ha='center', va='bottom')

    ax[1].set_title('Weekly Rotational Acceleration Analysis (Max and Average)')
    ax[1].set_xlabel('Week')
    ax[1].set_ylabel('Rotational Acceleration (g)')
    ax[1].legend()

    # Plot for On Plane Efficiency (average only)
    ax[2].plot(weekly_data['Date'], weekly_data['Avg On Plane Efficiency'], label='Avg On Plane Efficiency', color='purple', marker='o')

    for i, row in weekly_data.iterrows():
        color = 'green' if row['Avg On Plane Eff Change'] >= 2 else ('red' if row['Avg On Plane Eff Change'] <= -2 else 'purple')
        ax[2].plot(row['Date'], row['Avg On Plane Efficiency'], 'o', color=color)
        ax[2].text(row['Date'], row['Avg On Plane Efficiency'], f"{row['Avg On Plane Efficiency']:.1f}", fontsize=9, ha='center', va='bottom')

    ax[2].set_title('Weekly On Plane Efficiency Analysis (Average Only)')
    ax[2].set_xlabel('Week')
    ax[2].set_ylabel('On Plane Efficiency (%)')
    ax[2].legend()

    # Plot for Vertical Bat Angle and Attack Angle
    ax[3].plot(weekly_data['Date'], weekly_data['Avg Vertical Bat Angle'], label='Avg Vertical Bat Angle', color='green', marker='o')
    ax[3].plot(weekly_data['Date'], weekly_data['Avg Attack Angle'], label='Avg Attack Angle', color='blue', marker='s')

    for i, row in weekly_data.iterrows():
        color_vba = 'green' if -33 <= row['Avg Vertical Bat Angle'] <= -27 else 'red'
        ax[3].plot(row['Date'], row['Avg Vertical Bat Angle'], 'o', color=color_vba)
        ax[3].text(row['Date'], row['Avg Vertical Bat Angle'], f"{row['Avg Vertical Bat Angle']:.1f}", fontsize=9, ha='center', va='bottom')

        color_aa = 'green' if 10 <= row['Avg Attack Angle'] <= 12 else 'blue'
        ax[3].plot(row['Date'], row['Avg Attack Angle'], 's', color=color_aa)
        ax[3].text(row['Date'], row['Avg Attack Angle'], f"{row['Avg Attack Angle']:.1f}", fontsize=9, ha='center', va='bottom')

    ax[3].set_title('Weekly Vertical Bat Angle and Attack Angle Analysis')
    ax[3].set_xlabel('Week')
    ax[3].set_ylabel('Degrees')
    ax[3].legend()

    # Plot for Early Connection and Connection at Impact
    ax[4].plot(weekly_data['Date'], weekly_data['Avg Early Connection'], label='Avg Early Connection', color='green', marker='o')
    ax[4].plot(weekly_data['Date'], weekly_data['Avg Connection at Impact'], label='Avg Connection at Impact', color='blue', marker='s')

    for i, row in weekly_data.iterrows():
        color_ec = 'green' if 90 <= row['Avg Early Connection'] <= 100 else 'red'
        ax[4].plot(row['Date'], row['Avg Early Connection'], 'o', color=color_ec)
        ax[4].text(row['Date'], row['Avg Early Connection'], f"{row['Avg Early Connection']:.1f}", fontsize=9, ha='center', va='bottom')

        color_ci = 'green' if 80 <= row['Avg Connection at Impact'] <= 90 else 'blue'
        ax[4].plot(row['Date'], row['Avg Connection at Impact'], 's', color=color_ci)
        ax[4].text(row['Date'], row['Avg Connection at Impact'], f"{row['Avg Connection at Impact']:.1f}", fontsize=9, ha='center', va='bottom')

    # Set title and labels for Early Connection and Connection at Impact graph
    ax[4].set_title('Weekly Early Connection and Connection at Impact Analysis')
    ax[4].set_xlabel('Week')
    ax[4].set_ylabel('Degrees')
    ax[4].legend()

    # Format date on the x-axis for all graphs
    ax[0].xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
    ax[1].xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
    ax[2].xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
    ax[3].xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))
    ax[4].xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))

    # Rotate x-axis labels for better readability
    plt.setp(ax[0].xaxis.get_majorticklabels(), rotation=45)
    plt.setp(ax[1].xaxis.get_majorticklabels(), rotation=45)
    plt.setp(ax[2].xaxis.get_majorticklabels(), rotation=45)
    plt.setp(ax[3].xaxis.get_majorticklabels(), rotation=45)
    plt.setp(ax[4].xaxis.get_majorticklabels(), rotation=45)

    # Adjust layout and save the current figure to the PDF
    plt.tight_layout()
    pdf.savefig(fig)  # Save the figure to the PDF

    # Close the figure to free up memory
    plt.close()

# The generated PDF file will be saved at pdf_path
print(f'PDF saved at: {pdf_path}')


PDF saved at: /Users/ivanquackenbush/Documents/Blast/Steven_Schneider_updates.pdf
